# zhihu python API 

In [1]:
from zhihu import Question
from zhihu import Answer
from zhihu import User
from zhihu import Collection

In [2]:
# User 代表一个用户，处理用户相关操作。创建一个 User 对象需传入该用户的 url ，如：
user_url = "https://www.zhihu.com/people/qichenguang"
user = User(user_url)

In [3]:
collections_num = user.get_collections_num()

In [4]:
collections_num

40

In [ ]:
# 获取该用户的收藏夹
collections = user.get_collections()

In [ ]:
for collection in collections:
    print collection.get_name()
    dir_name = collection.get_name()
    # 得到该收藏夹下的前十个回答
    #top_answers = collection.get_top_i_answers(10)
    # 获取该收藏夹下的所有答案
    answers = collection.get_all_answers()
    # 把答案内容转成txt，markdown
    for answer in answers:
        answer.to_txt_with_dir(dir_name)

In [ ]:
print str(name).decode('gbk', 'ignore')

# jieba pathon API

In [ ]:
import jieba

In [ ]:
# 搜索引擎模式
seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  

In [ ]:
dict(seg_list)

In [ ]:
chilist = list(seg_list)

In [ ]:
chilist

# Index

In [5]:
import os
import jieba
import string
from pysqlite2 import dbapi2 as sqlite

In [6]:
rootdir = 'E:/project/pychram/zhihu/zhihu-python/text'

In [7]:
# Create a list of words to ignore
ignorewords={'the':1,'of':1,'to':1,'and':1,'a':1,'in':1,'is':1,'it':1}

In [8]:
class crawler:
    # Initialize the crawler with the name of database
    def __init__(self,dbname):
        self.con=sqlite.connect(dbname)

    def __del__(self):
        self.con.close()

    def dbcommit(self):
        self.con.commit()

    # Auxilliary function for getting an entry id and adding 
    # it if it's not present
    def getentryid(self,table,field,value,createnew=True):
        cur = self.con.execute("select rowid from %s where %s='%s'" % (table,field,value))
        res = cur.fetchone()
        if res == None:
            cur = self.con.execute("insert into %s (%s) values ('%s')" % (table,field,value))
            return cur.lastrowid,True
        else:
            return res[0],False

    # Index an individual page
    def addtoindex(self,file_class,title,author,zhan_num,url,content):
        #
        if self.isindexed(url):
            print 'already had indexing :' + url
            return
        print 'Indexing ' + url
  
        #
        try:
            words = self.separatewords(title + " " + author + " " + content)
            #print 'words:',words
        except Exception,ex:
            print "separatewords() error",ex
        #    
        print 'len(content):',len(content)
        print 'len(words):',len(words)        
        # Get the URL id
        urlid,createNew = self.getentryid('urllist','url',url)
        if createNew == True:
            cur = self.con.execute("insert into urldetail(urlid,title,author,zhan_num) values (%d,'%s','%s',%d)" \
                                   % (urlid,str(title).decode('gbk', 'ignore'),str(author).decode('gbk', 'ignore'),zhan_num))
    
        # Link each word to this url
        for i in range(len(words)):
            word = words[i]
            if word in ignorewords: continue
            wordid,creatNew = self.getentryid('wordlist','word',word)
            self.con.execute("insert into wordlocation(urlid,wordid,location) values (%d,%d,%d)" % (urlid,wordid,i))

        
    # Seperate the words by any non-whitespace character
    def separatewords(self,text):
        # 搜索引擎模式
        seg_list = jieba.cut_for_search(text)  
        return list(seg_list)

    
    # Return true if this url is already indexed
    def isindexed(self,url):
        u = self.con.execute("select rowid from urllist where url='%s'" % url).fetchone()
        if u != None:
            #检查它是否已经被检索过了
            v = self.con.execute('select * from wordlocation where urlid=%d ' % u[0]).fetchone()
            if v != None: return True
        return False
  
    # Add a link between two pages
    def addlinkref(self,urlFrom,urlTo,linkText):
        words = self.separatewords(linkText)
        fromid,_ = self.getentryid('urllist','url',urlFrom)
        toid,_ = self.getentryid('urllist','url',urlTo)
        if fromid == toid: return
        cur = self.con.execute("insert into link(fromid,toid) values (%d,%d)" % (fromid,toid))
        linkid = cur.lastrowid
        for word in words:
            if word in ignorewords: continue
            wordid,_ = self.getentryid('wordlist','word',word)
            self.con.execute("insert into linkwords(linkid,wordid) values (%d,%d)" % (linkid,wordid))

    # Starting with a list of pages, do a breadth
    # first search to the given depth, indexing pages
    # as we go
    def crawl(self,dir_path):
        i = 0
        for parent,dirnames,filenames in os.walk(rootdir):  
            #输出文件信息
            for filename in filenames:                        
                #print "parent is:" + parent
                #print type(parent)
                #print "filename is: " + str(filename).decode('gbk', 'ignore')
                #输出文件路径信息
                #print "the full name of the file is:" + os.path.join(parent,filename) 
                file_class = parent.split("\\")[1]
                #print file_class
                full_name = os.path.join(parent,filename) 
                i += 1
                print str(i) + " ,full_name is " + str(full_name).decode('gbk', 'ignore')
                # 读取文件
                title,author,zhan_num,url,content = self.read_and_parse_file(full_name)
                if title == None:
                    continue
                #
                self.addtoindex(file_class,title,author,zhan_num,url,content) 
                self.dbcommit()

    
    def read_and_parse_file(self,file_path):
        # 去掉标点符号
        delset = string.punctuation
        try:
            if os.path.exists(file_path):
                f = open(file_path, "r")
                lines = f.readlines()
                f.close()
                if len(lines) < 5:
                    return None,None,0,None,None
                title = lines[0].strip()
                fields = lines[2].split(':')
                zhan_num = int(fields[2].strip())
                author = fields[1].strip().split(' ')[0].strip()
                content = ''.join(lines[5:-2])
                url = lines[-1].split(' ')[1].strip()
                #
                title = title.translate(None,delset)
                author = author.translate(None,delset)
                content = content.translate(None,delset)
                return title,author,zhan_num,url,content
        except IOError:
            error_file_path = file_path
            print "error_file_path:" , error_file_path
        
            
    # Create the database tables
    def createindextables(self):
        self.con.execute('create table urllist(url)')
        self.con.execute('create table urldetail(urlid,title,author,zhan_num)')
        self.con.execute('create table wordlist(word)')
        self.con.execute('create table wordlocation(urlid,wordid,location)')
        self.con.execute('create table link(fromid integer,toid integer)')
        self.con.execute('create table linkwords(wordid,linkid)')
        self.con.execute('create index wordidx on wordlist(word)')
        self.con.execute('create index urlidx on urllist(url)')
        self.con.execute('create index wordurlidx on wordlocation(wordid)')
        self.con.execute('create index urltoidx on link(toid)')
        self.con.execute('create index urlfromidx on link(fromid)')
        self.dbcommit()      

In [9]:
file_dir_list = [ 'E:/project/pychram/zhihu/zhihu-python/text']
crawlerObj = crawler('search_zhihu_index.db')

In [ ]:
crawlerObj.createindextables()

In [ ]:
crawlerObj.crawl(file_dir_list)

In [ ]:
[row for row in crawlerObj.con.execute('select urlid from wordlocation where wordid=500') ]

In [10]:
class searcher:
    def __init__(self,dbname):
        self.con=sqlite.connect(dbname)

    def __del__(self):
        self.con.close()
        
    def getmatchrows(self,q):
        # Strings to build the query
        fieldlist='w0.urlid'
        tablelist=''  
        clauselist=''
        wordids=[]

        # Split the words by spaces
        words = q.split(' ')  
        tablenumber = 0

        for word in words:
            # Get the word ID
            wordrow = self.con.execute("select rowid from wordlist where word='%s'" % word).fetchone()
            if wordrow != None:
                wordid = wordrow[0]
                wordids.append(wordid)
                if tablenumber > 0:
                    tablelist += ','
                    clauselist += ' and '
                    clauselist += 'w%d.urlid = w%d.urlid and ' % (tablenumber - 1,tablenumber)
                fieldlist += ',w%d.location' % tablenumber
                tablelist += 'wordlocation w%d' % tablenumber      
                clauselist += 'w%d.wordid = %d' % (tablenumber,wordid)
                tablenumber+=1

        # Create the query from the separate parts
        fullquery = 'select %s from %s where %s' % (fieldlist,tablelist,clauselist)
        print fullquery
        cur = self.con.execute(fullquery)
        rows=[row for row in cur]

        return rows,wordids   
    
    def getscoredlist(self,rows,wordids):
        totalscores = dict([(row[0],0) for row in rows])

        # This is where we'll put our scoring functions
        weights = [(1.0,self.frequencyscore(rows)),
                   (1.0,self.locationscore(rows)),
                   (1.0,self.distancescore(rows)),
                   (1.0,self.inboundlinkscore(rows)),
                   (1.0,self.linktextscore(rows,wordids))]
        #
        for (weight,scores) in weights:
            for url in totalscores:
                totalscores[url] += weight * scores[url]
        return totalscores

    def geturlname(self,id):
        return self.con.execute("select url from urllist where rowid=%d" % id).fetchone()[0]
    
    def geturldetail(self,id):
        return list(self.con.execute("select title,author,zhan_num from urldetail where urlid=%d" % id).fetchone())
    
    def getupperzhannumer(self,num,order='desc'):
        cur = self.con.execute("select url,title,author,zhan_num from urldetail,urllist \
                                 where urldetail.urlid = urllist.rowid \
                                 and urldetail.zhan_num >= %d order by urldetail.zhan_num %s" % (num,order))
        rows=[ row for row in cur]
        
        return rows

    def query(self,q):
        rows,wordids = self.getmatchrows(q)
        scores = self.getscoredlist(rows,wordids)
        rankedscores=[(score,url) for (url,score) in scores.items()]
        rankedscores.sort()
        rankedscores.reverse()
        for (score,urlid) in rankedscores[0:100]:
            print '%f\t%s' % (score,self.geturlname(urlid))
            urldetail = list(self.geturldetail(urlid))
            print '%s,%s,%d' % (urldetail[0],urldetail[1],urldetail[2])
        return wordids,[r[1] for r in rankedscores[0:100]]

    def normalizescores(self,scores,smallIsBetter=0):
        vsmall = 0.00001 # Avoid division by zero errors
        if smallIsBetter:
            minscore = min(scores.values())
            return dict([(u,float(minscore)/max(vsmall,l)) for (u,l) in scores.items()])
        else:
            maxscore = max(scores.values())
            if maxscore == 0: maxscore = vsmall
            return dict([(u,float(c)/maxscore) for (u,c) in scores.items()])        
        
    def frequencyscore(self,rows):
        counts = dict([(row[0],0) for row in rows])
        for row in rows: counts[row[0]] += 1
        return self.normalizescores(counts)  
    
    def locationscore(self,rows):
        locations = dict([(row[0],1000000) for row in rows])
        for row in rows:
            loc = sum(row[1:])
            if loc < locations[row[0]]: locations[row[0]] = loc
    
        return self.normalizescores(locations,smallIsBetter=1) 
    
    def distancescore(self,rows):
        # If there's only one word, everyone wins!
        if len(rows[0]) <= 2: return dict([(row[0],1.0) for row in rows])

        # Initialize the dictionary with large values
        mindistance = dict([(row[0],1000000) for row in rows])

        for row in rows:
            dist = sum([abs(row[i]-row[i-1]) for i in range(2,len(row))])
            if dist < mindistance[row[0]]: mindistance[row[0]] = dist
                
        return self.normalizescores(mindistance,smallIsBetter=1)    

    def inboundlinkscore(self,rows):
        uniqueurls = dict([(row[0],1) for row in rows])
        inboundcount = dict([(u, self.con.execute('select count(*) from link where toid=%d' % u).fetchone()[0]) for u in uniqueurls])   
        return self.normalizescores(inboundcount)
    
    def pagerankscore(self,rows):
        pageranks=dict([(row[0],self.con.execute('select score from pagerank where urlid=%d' % row[0]).fetchone()[0]) for row in rows])
        maxrank = max(pageranks.values())
        normalizedscores = dict([(u,float(l)/maxrank) for (u,l) in pageranks.items()])
        return normalizedscores    
    
    def linktextscore(self,rows,wordids):
        vsmall = 0.00001 # Avoid division by zero errors
        linkscores = dict([(row[0],0) for row in rows])
        for wordid in wordids:
            cur = self.con.execute('select link.fromid,link.toid from linkwords,link where wordid=%d and linkwords.linkid=link.rowid' % wordid)
            for (fromid,toid) in cur:
                if toid in linkscores:
                    pr = self.con.execute('select score from pagerank where urlid=%d' % fromid).fetchone()[0]
                    linkscores[toid] += pr
        maxscore = max(max(linkscores.values()),vsmall)
        normalizedscores = dict([(u,float(l)/maxscore) for (u,l) in linkscores.items()])
        return normalizedscores  

In [11]:
searcherObj = searcher('search_zhihu_index.db')

In [12]:
searcherObj.getmatchrows('玄不救')

([(199, 10),
  (210, 10),
  (216, 17),
  (311, 12),
  (313, 15),
  (322, 13),
  (326, 32),
  (335, 20),
  (337, 6),
  (344, 12),
  (346, 16),
  (474, 17),
  (580, 1287),
  (584, 27)],
 [43313])

In [ ]:
searcherObj.query('''青虾''')

In [ ]:
searcherObj.query("快播")

In [ ]:
searcherObj.geturlname(140)

In [ ]:
searcherObj.geturldetail(140)

In [ ]:
urldetail = list(searcherObj.geturldetail(140))

In [ ]:
print urldetail[0]

In [ ]:
print urldetail[1]

In [ ]:
print urldetail[2]

In [5]:
rows = searcherObj.getupperzhannumer(0,'asc')

In [7]:
i = 0
for row in rows:
    i +=1
    print i,row[0]
    print row[1],row[2],row[3]

1 http://www.zhihu.com/question/20772002/answer/21152078
有哪些工具或者软件堪称神器？ RAY 0
2 http://www.zhihu.com/question/20772002/answer/29444209
有哪些工具或者软件堪称神器？ wangming 0
3 http://www.zhihu.com/question/38765074/answer/77975727
生物为什么会有雌雄之分？ Serena 0
4 http://www.zhihu.com/question/25654406/answer/31467157
上班的人怎样自学英语？ 茶茶 0
5 http://www.zhihu.com/question/27044578/answer/35079738
有哪些值得推荐的关于英文词根的好书？ 袁飞 0
6 http://www.zhihu.com/question/24590883/answer/77038690
哪些 Python 库让你相见恨晚？ Phil 1
7 http://www.zhihu.com/question/20772002/answer/16895560
有哪些工具或者软件堪称神器？ Bob 1
8 http://www.zhihu.com/question/20772002/answer/65782245
有哪些工具或者软件堪称神器？ 茤i 1
9 http://www.zhihu.com/question/28057213/answer/44238642
历史上有哪些打脸的故事？ 匿名用户 1
10 http://www.zhihu.com/question/35417187/answer/70583375
为什么有些国家为了民主和更好的生活而革命，却反而陷入困苦？ 君雅 1
11 http://www.zhihu.com/question/39507724/answer/89807287
如何看待“男子勒死自闭儿子埋后山 ”这一新闻？ 自由谬误等级 1
12 http://www.zhihu.com/question/27044578/answer/71757698
有哪些值得推荐的关于英文词根的好书？ weidong 1
13 http://www.zhihu